In [ ]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("API_KEY")

API_HEADERS = {"X-ELS-APIKey": API_KEY}

def fetch_journals():
    total_records = 1000
    query = 'a'
    count = 25
    start = 0
    pubyear_filter = "PUBYEAR > 1999 AND PUBYEAR < 2024"
    full_query = f"{query} AND {pubyear_filter}"
    articles = []
    
    for start in range(0, total_records, count):
        # Make API request to Scopus
        scopus_url = "https://api.elsevier.com/content/search/scopus"
        
        params = {"query": full_query, "count": count, "start": start}
        response = requests.get(scopus_url, headers=API_HEADERS, params=params)

        if response.status_code == 200:
            try:
                # Get the data
                search_results = response.json().get('search-results', {}).get('entry', [])
                
                for result in search_results:
                    #Match only useful data
                    title = result.get('dc:title', 'No title available')
                    publication_name = result.get('dc:creator', 'No publication name')
                    publication_date = result.get('prism:coverDate', 'No date available')
                    affiliations = result.get('affiliation', [])
                    universities = ", ".join([aff.get('affilname', 'Unknown university') for aff in affiliations])
                    abstract = result.get('prism:doi', 'No title available')
                    link = f'https://doi.org/{abstract}'
                    
                    articles.append({
                        "title": title,
                        "date": publication_date,
                        "universities": universities,
                        "author": publication_name,
                        "abstract": link
                    })
                
            except ValueError:
                print(f"Error: Invalid JSON response from API, content: {response.text}")
        else:
            print(f"Error: API request failed, status code: {response.status_code}, content: {response.text}")
            
    # Convert articles to DataFrame
    df = pd.DataFrame(articles)
    display(df)
    
    # Save to CSV
    #df.to_csv("journals.csv", index=False)
    #print("CSV file 'journals.csv' has been created.")

if __name__ == "__main__":
    fetch_journals()


,title,date,universities,author,abstract
0,"(IM)PIETY: ISLAM, PIOUS SINNING AND REGULATED ...",2023-12-31,University of Canterbury,Al-Soukkary W.,https://doi.org/10.55831/ajis.v8i3.627
1,KNOWLEDGE OF THE MILKY WAY IN THE ARABIC CULTU...,2023-12-31,Universität zu Köln,Eckart A.,https://doi.org/10.55831/ajis.v8i3.637
2,WEAVING MODERNITY IN SALAFISM: A COMPARATIVE S...,2023-12-31,Faculty of Islamic Studies,Nasir M.M.,https://doi.org/10.55831/ajis.v8i3.619
3,PRISON LIFE IN THE EYES OF BEDIÜZZAMAN SAID NU...,2023-12-31,Australian Catholic University,Albayrak I.,https://doi.org/10.55831/ajis.v8i3.609
4,CRITICAL REFLECTIONS ON CURRENT STATUS OF SCHO...,2023-12-31,Charles Sturt University,Hussein Rassool G.,https://doi.org/10.55831/ajis.v8i3.641
...,...,...,...,...,...
995,"ASSESSMENT OF KNOWLEDGE, ATTITUDE, AND PRACTIC...",2023-12-31,Government Medical College Nagpur,Dudhe B.,https://doi.org/10.18231/j.joapr.2023.11.5.26.33
996,Religious Values and Food Waste: A Comparative...,2023-12-31,Czech University of Life Sciences Prague,Špička J.,https://doi.org/10.23762/FSO_VOL11_NO4_5
997,DETECTION OF THE BK VIRUS IN POST-TRANSPLANT P...,2023-12-31,Swargiya PNKS Government P.G. College,Choudhary V.,https://doi.org/10.18231/j.joapr.2023.11.5.64.69
998,Silvas publicas depopulatus erat (Cic. Mil. 26...,2023-12-31,Sapienza Università di Roma,La Bua G.,https://doi.org/10.13135/2532-5353/9338
